In [1]:
import pandas as pd
import scanpy as sc
import os
import numpy as np

In [5]:
datadir = '/home/jing/Phd_project/project_UCD_blca/blca_publication_OUTPUT/'
os.chdir(datadir)

In [8]:
mtx =os.path.join('blca_publication_OUTPUT_sct/',"sct_corrected_UMI.mtx")
cells=pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/',"sct_corrected_UMI_cells.txt"),header=None,index_col=0)
features=pd.read_csv(os.path.join('blca_publication_OUTPUT_sct/','sct_corrected_UMI_genes.txt'),header=None,sep='\t',index_col=0)
adata = sc.read_mtx(mtx)


In [10]:
adata= adata.T

In [11]:
adata.obs['CellID']= cells.index.tolist()
adata.var['Gene']= features.index.tolist()
adata.var.index= adata.var['Gene']
display(adata)

AnnData object with n_obs × n_vars = 15954 × 19647
    obs: 'CellID'
    var: 'Gene'

In [37]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata,base=2)

In [26]:
adata.obs['CellID'].str.split('_')

0        [GSM5288668, AAACCCACACTACTTT-1]
1        [GSM5288668, AAACCCATCGGTCTGG-1]
2        [GSM5288668, AAACGAACACACACTA-1]
3        [GSM5288668, AAACGAAGTGTTACAC-1]
4        [GSM5288668, AAACGCTCACAATGCT-1]
                       ...               
15949    [GSM5288674, TTTGATCGTACGAGCA-1]
15950    [GSM5288674, TTTGATCGTTGCTGAT-1]
15951    [GSM5288674, TTTGGAGCACCTTCCA-1]
15952    [GSM5288674, TTTGGAGCAGACAAAT-1]
15953    [GSM5288674, TTTGGTTGTACACGCC-1]
Name: CellID, Length: 15954, dtype: object

In [29]:
adata.obs['Sample'] = adata.obs['CellID'].str.split('_').str[0]
adata.obs.index = adata.obs['CellID']

In [32]:
adata.obs['Type'] = 'N/A'

In [34]:
for i in ['GSM5288668', 'GSM5288669']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'NMIBC'
for i in ['GSM5288670', 'GSM5288671']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'MIBC'    
for i in ['GSM5288672', 'GSM5288674']:
    adata.obs.loc[adata.obs['Sample'].str.contains(i, na=False), 'Type'] = 'Healthy'    

In [35]:
from gseapy import Biomart

In [36]:
bm = Biomart()
# note the dataset and attribute names are different
m2h = bm.query(dataset='mmusculus_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'hsapiens_homolog_ensembl_gene',
                           'hsapiens_homolog_associated_gene_name'])

h2m = bm.query(dataset='hsapiens_gene_ensembl',
               attributes=['ensembl_gene_id','external_gene_name',
                           'mmusculus_homolog_ensembl_gene',
                           'mmusculus_homolog_associated_gene_name'])

In [79]:
len(chunk)

1000

In [116]:
chunk_size = 1000

gene_list = list(adata.var['Gene'])  # Convert to list if not already
chunks = [gene_list[i:i + chunk_size] for i in range(0, 3000, chunk_size)]

In [123]:
len(gene_list)

19647

In [139]:
result = bm.query(dataset='mmusculus_gene_ensembl',
                       attributes=[ 'external_gene_name',
                                   'hsapiens_homolog_associated_gene_name'],
                       filters={'external_gene_name': gene_list[19000:] })

In [ ]:
result = bm.query(dataset='mmusculus_gene_ensembl',
                       attributes=[ 'external_gene_name',
                                   'hsapiens_homolog_associated_gene_name'],
                       filters={'external_gene_name': gene_list[18000:19000] })

In [140]:
 #final=pd.concat([results,result],ignore_index=True)
final=pd.concat([final,result],ignore_index=True)

In [141]:
final

,ensembl_gene_id,external_gene_name,hsapiens_homolog_ensembl_gene,hsapiens_homolog_associated_gene_name
0,ENSMUSG00000101321,Gm28817,NaN,NaN
1,ENSMUSG00000101170,Gm29343,NaN,NaN
2,ENSMUSG00000100231,Gm29554,ENSG00000139351,SYCP3
3,ENSMUSG00000042372,Dmrt3,ENSG00000064218,DMRT3
4,ENSMUSG00000114511,0710001A04Rik,NaN,NaN
...,...,...,...,...
17596,NaN,Gm14319,NaN,NaN
17597,NaN,1810012K08Rik,NaN,NaN
17598,NaN,Platr3,NaN,NaN
17599,NaN,Gm13262,NaN,NaN


In [142]:
# Map human homologs back to adata.var
mouse_to_human = dict(zip(final['external_gene_name'], final['hsapiens_homolog_associated_gene_name']))
adata.var['Human_Gene'] = adata.var['Gene'].map(mouse_to_human)

# Display a preview
adata.var[['Gene', 'Human_Gene']].head()

,Gene,Human_Gene
Gene,,
Xkr4,Xkr4,NaN
Sox17,Sox17,NaN
Mrpl15,Mrpl15,NaN
Lypla1,Lypla1,NaN
Gm37988,Gm37988,NaN


In [145]:
len(set(adata.var['Human_Gene']))

12949

In [ ]:
#chunks = [gene_list[i:i + chunk_size] for i in range(3000, 6000, chunk_size)]
chunks = gene_list[18000:19674] 

for chunk in chunks:
    print(chunk)
    result = bm.query(dataset='mmusculus_gene_ensembl',
                       attributes=[ 'external_gene_name',
                                   'hsapiens_homolog_associated_gene_name'],
                       filters={'external_gene_name': chunk})
    #final=pd.concat([results,result],ignore_index=True)
    final=pd.concat([final,result],ignore_index=True)

In [114]:
# Map human homologs back to adata.var
mouse_to_human = dict(zip(results_df['external_gene_name'], results_df['hsapiens_homolog_associated_gene_name']))
adata.var['Human_Gene'] = adata.var['Gene'].map(mouse_to_human)

# Display a preview
adata.var[['Gene', 'Human_Gene']].head()

,ensembl_gene_id,external_gene_name,hsapiens_homolog_ensembl_gene,hsapiens_homolog_associated_gene_name
0,ENSMUSG00000101321,Gm28817,NaN,NaN
1,ENSMUSG00000101170,Gm29343,NaN,NaN
2,ENSMUSG00000100231,Gm29554,ENSG00000139351,SYCP3
3,ENSMUSG00000042372,Dmrt3,ENSG00000064218,DMRT3
4,ENSMUSG00000114511,0710001A04Rik,NaN,NaN
...,...,...,...,...
1562,NaN,Gata5os,NaN,NaN
1563,NaN,Gm16316,NaN,NaN
1564,NaN,Zfp335os,NaN,NaN
1565,NaN,9130015L21Rik,NaN,NaN


In [125]:
final

,ensembl_gene_id,external_gene_name,hsapiens_homolog_ensembl_gene,hsapiens_homolog_associated_gene_name
0,ENSMUSG00000101321,Gm28817,NaN,NaN
1,ENSMUSG00000101170,Gm29343,NaN,NaN
2,ENSMUSG00000100231,Gm29554,ENSG00000139351,SYCP3
3,ENSMUSG00000042372,Dmrt3,ENSG00000064218,DMRT3
4,ENSMUSG00000114511,0710001A04Rik,NaN,NaN
...,...,...,...,...
15956,NaN,Gm13652,NaN,NaN
15957,NaN,Gm13816,NaN,NaN
15958,NaN,6430710C18Rik,NaN,NaN
15959,NaN,Gm13404,NaN,NaN


In [43]:
m2h[m2h['external_gene_name'] =='Xkr4']

,ensembl_gene_id,external_gene_name,hsapiens_homolog_ensembl_gene,hsapiens_homolog_associated_gene_name
44896,ENSMUSG00000051951,Xkr4,ENSG00000206579,XKR4


In [ ]:
#For raw data retrival 
def subset(data,name):
    mtx =f"{data}_filtered_matrix.mtx.gz"
    cells=pd.read_csv(f"{data}_filtered_barcodes.tsv.gz",header=None)
    features=pd.read_csv(f'{data}_filtered_features.tsv.gz',header=None,sep='\t')
    adata = sc.read_mtx(mtx)
    
    adata= adata.T
    print(adata)
    adata.obs['CellID']= cells[0].tolist()
    adata.var['Gene']= features[1].tolist()
    adata.var.index= adata.var['Gene']
    
    adata.var_names_make_unique()
    barcodes_subset = barcodes_uro.loc[barcodes_uro['Sample'] == name, 'Barcode']
    adata_subset = adata[adata.obs['CellID'].isin(barcodes_subset)]
    print(adata_subset)
    adata_subset = adata_subset[:, adata_subset.var_names.isin(gene_list)]
    sc.pp.normalize_total(adata_subset, target_sum=1e4)
    sc.pp.log1p(adata_subset,base=2)
    adata_subset_df=adata_subset.to_df()
   # adata_subset_df.to_pickle(f"{name}_sub.pkl")
